# GCN and GAT IMPLEMENTATION FOR GRAPH EMBEDDING

## Load necessary libraries

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 36.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## Load Data

In [ ]:
input_edges = np.loadtxt("/content/drive/MyDrive/yr4 Design Project/final/final_adjacency_edgeindex.txt")
input_edges = torch.FloatTensor(input_edges)
edge_index = input_edges.nonzero().t().contiguous()
print(type(edge_index))
print(edge_index)

<class 'torch.Tensor'>
tensor([[   0,    1,    2,  ..., 1545, 1545, 1545],
        [1542, 1508, 1508,  ..., 1206, 1211, 1253]])


In [ ]:
m = np.loadtxt("/content/drive/MyDrive/yr4 Design Project/final/microbe-microbe.txt")
print(np.sum(m))

138.0


In [ ]:
d = np.loadtxt("/content/drive/MyDrive/yr4 Design Project/final/drug-drug.txt")
print(np.sum(d))

5586.0


In [ ]:
a = np.loadtxt("/content/drive/MyDrive/yr4 Design Project/final/Adj_transpose.txt")
print(np.sum(a))
# holds links between microbe-disease-drug as well as known mdad microbe-drug
count=0
for i in range(0, len(a[0])):
  for j in range(0, len(a[1])):
    if a[i][j] > 1:
      print(".", sep="")
      count+=1
print(count)

2579.0
0


In [ ]:
print(edge_index.shape)
print(edge_index)

torch.Size([2, 10882])
tensor([[   0,    1,    2,  ..., 1545, 1545, 1545],
        [1542, 1508, 1508,  ..., 1206, 1211, 1253]])


In [ ]:
x = np.loadtxt("/content/drive/MyDrive/yr4 Design Project/final/x_new.txt")
print(np.sum(x)) # number of nodes, number of features
count = 0
for i in x.flatten():
  if i != 0:
    count+=1
    # print(i)
print(count)

188384396101137.8
3830116


In [ ]:
import scipy.sparse as sp
def normalize_features(feat):

    degree = np.asarray(feat.sum(1)).flatten()

    # set zeros to inf to avoid dividing by zero
    degree[degree == 0.] = np.inf
    degree_inv = 1. / degree
    degree_inv_mat = sp.diags([degree_inv], [0])
    feat_norm = degree_inv_mat.dot(feat)

    return feat_norm

In [ ]:

x_norm = torch.FloatTensor(normalize_features(x))
indices = torch.nonzero(x_norm)
values = x_norm[indices[:, 0], indices[:, 1]]
# Create sparse feature matrix
feature_size = x_norm.size()
feature_matrix = torch.sparse.FloatTensor(indices.t(), values, feature_size)


In [ ]:
# # data.y will be 2D [0 A, A' 0]
y = np.vstack((np.hstack((np.zeros((1373, 1373)), a)), np.hstack((np.transpose(a), np.zeros((173, 173))))))

In [ ]:

import scipy.sparse as sp
def compute_symmetric_normalized_laplacian(adj):
    # Compute the degree matrix

    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.0
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

    # Compute the symmetric Laplacian
    laplacian =  d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
    return laplacian.toarray()



In [ ]:
# Normalize y

z = compute_symmetric_normalized_laplacian(y)
y = torch.FloatTensor(z)

In [ ]:
from torch_geometric.data import Data
import torch_geometric.transforms as T

data = Data(x=feature_matrix, edge_index=edge_index, y = y)

print(data)

Data(x=[1546, 3092], edge_index=[2, 10882], y=[1546, 1546])


In [ ]:
print(data.num_node_features)

3092


In [ ]:
print(data.x.shape, data.edge_index.shape)

torch.Size([1546, 3092]) torch.Size([2, 10882])


In [ ]:
print((data.x).coalesce().indices())
print(data.edge_index)
print(data.y)

tensor([[   0,    0,    0,  ..., 1545, 1545, 1545],
        [   0,    1,    2,  ..., 3089, 3090, 3091]])
tensor([[   0,    1,    2,  ..., 1545, 1545, 1545],
        [1542, 1508, 1508,  ..., 1206, 1211, 1253]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


# Define class (functions) as well as NN architecture

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv

class InnerProductDecoder(nn.Module):
    def __init__(self, dropout=0.0):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout

    def forward(self, inputs):
        inputs = F.dropout(inputs, p=self.dropout, training=self.training)
        x = torch.transpose(inputs, 0, 1)
        x = torch.matmul(inputs, x)
        # x = torch.flatten(x)  # we dont want it flatted due to computation of kl loss
        output = F.sigmoid(x)  #Graph2MDA used sigmoid function
        return x

# class SparseConv(nn.Module):
#     def __init__(self, input_dim, output_dim, dropout, adj, act=torch.relu):
#         super(SparseConv, self).__init__()
#         self.input_dim = input_dim
#         self.output_dim = output_dim
#         self.dropout = dropout
#         self.adj = adj
#         self.act = act
#         self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
#         self.reset_parameters()

#     def reset_parameters(self):
#         nn.init.xavier_uniform_(self.weight)

#     def forward(self, inputs):
#         x = inputs.to_dense()
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = torch.mm(x, self.weight)
#         x = torch.sparse.mm(self.adj, x)
#         outputs = self.act(x)
#         return outputs

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # self.sparse = SparseLayer(x, 1)
        # self.sparse = SparseConv(data.num_node_features, 800, 0.4, input_edges)
        self.conv1 = GCNConv(data.num_node_features, 800)  # input_channels = number of nodes, output_channels = number of nodes again
        self.conv2 = GCNConv(800, 173)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # x = self.sparse(x)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p = 0.4, training=self.training)
        x = self.conv2(x, edge_index)

        return x


In [ ]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 800
        self.in_head = 1
        self.out_head = 1


        self.conv1 = GATConv(data.num_features, self.hi d, heads=self.in_head, dropout=0.4)
        self.conv2 = GATConv(self.hid*self.in_head, 173, concat=False,
                             heads=self.out_head, dropout=0.4)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.4, training=self.training)
        x = self.conv2(x, edge_index)

        return x

In [ ]:
print(data.num_node_features)
print(x_norm)

3092
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.4481e-12, 4.6694e-17,
         7.2605e-19],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.7377e-12, 4.9734e-17,
         7.7329e-19],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.1085e-11, 2.4983e-16,
         5.9075e-18],
        ...,
        [3.2826e-11, 3.2826e-11, 3.2826e-11,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.2177e-11, 2.2177e-11, 2.2177e-11,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.5402e-09, 2.5402e-09, 2.5402e-09,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])


## Run models

In [ ]:
loss_values = list()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
loss_obj = nn.MSELoss()
reconstructions = InnerProductDecoder()

model.train()



for epoch in range(400):
    optimizer.zero_grad()
    out = model(data) # embedding
    op = reconstructions(out)
    print("Training ", epoch)

    loss = loss_obj(op, data.y)

    loss_values.append(loss.item())
    loss.backward()
    optimizer.step()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)

loss_obj = nn.MSELoss()
reconstructions = InnerProductDecoder()


model.train()



for epoch in range(400):
    optimizer.zero_grad()
    out = model(data) # embedding
    op = reconstructions(out)

    loss = loss_obj(op, data.y)

    loss_values.append(loss.item())
    loss.backward()
    optimizer.step()

In [ ]:
print(op)
print(data.y)
print(loss_obj(op, data.y).item())

tensor([[ 0.0010,  0.0010,  0.0010,  ..., -0.0004,  0.0004,  0.0004],
        [ 0.0010,  0.0010,  0.0010,  ..., -0.0004,  0.0004,  0.0004],
        [ 0.0010,  0.0010,  0.0010,  ..., -0.0004,  0.0004,  0.0004],
        ...,
        [-0.0004, -0.0004, -0.0004,  ...,  0.0015,  0.0002, -0.0001],
        [ 0.0004,  0.0004,  0.0004,  ...,  0.0002,  0.0006,  0.0003],
        [ 0.0004,  0.0004,  0.0004,  ..., -0.0001,  0.0003,  0.0011]],
       grad_fn=<MmBackward0>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
6.373337237164378e-05


In [ ]:
print(out)
print(out.shape)

tensor([[-0.0024,  0.0038, -0.0015,  ..., -0.0009, -0.0038, -0.0017],
        [-0.0024,  0.0038, -0.0015,  ..., -0.0009, -0.0038, -0.0017],
        [-0.0024,  0.0038, -0.0015,  ..., -0.0009, -0.0038, -0.0017],
        ...,
        [-0.0014, -0.0045, -0.0011,  ...,  0.0032,  0.0067,  0.0008],
        [ 0.0005,  0.0025, -0.0016,  ..., -0.0051,  0.0017, -0.0016],
        [ 0.0024,  0.0009, -0.0024,  ..., -0.0035, -0.0032,  0.0001]],
       grad_fn=<AddBackward0>)
torch.Size([1546, 173])


In [ ]:
output = out.detach().numpy()
np.savetxt("/content/gat_newx_400_22_lowloss.txt", output)

In [ ]:
np.savetxt("/content/loss_gcn_lowloss.txt", np.array(loss_values))

## Evaluation

In [ ]:
new_emb = np.loadtxt("gat_newx_400_22_lowloss.txt")
new_adj = np.loadtxt("Adj_transpose_anushka.txt")
print(new_emb.shape)
print(new_adj.shape)
#print(np.sum(new_adj,axis=0))
labels = new_adj
k_folds = k_folds
# split data into train and test
num_test = int(np.floor(labels.shape[0] / k_folds))
num_train = labels.shape[0] - num_test
embeddings = new_emb

all_idx = list(range(labels.shape[0]))#:only drug ids
print(labels.shape[0])
print(all_idx)#0-1372
print(num_test)#137
print(num_train)#1236
perf_list = []
print("###################################")
print(f'features:{features.shape}')#1546,3092
print(f'embedding:{embeddings.shape}')#1546,173
print(f'labels:{labels.shape}')#1373,173

for k in range(k_folds):
    print("------this is %dth cross validation------" % (k + 1))
    np.random.seed(k)
    np.random.shuffle(all_idx)#shuffle ids of drugs

    train_idx = all_idx[:num_train]#0:1236
    test_idx = all_idx[num_train:(num_train + num_test)]#1236-1372

    Y_train = labels[train_idx]

    print(Y_train.shape)#1236,173
    Y_test = labels[test_idx]
    X_train = embeddings[train_idx]
    X_test = embeddings[test_idx]
    print(f'X_train:{X_train.shape}')
    print(f'Y_train:{Y_train.shape}')
    print(f'X_test:{X_test.shape}')

    print('....train_nn....')
    y_score = train_nn(X_train, Y_train, X_test, Y_test)
    print(f'Predicted prob:{y_score.shape}')
    print("---------------------------------")
    print('....evalution....')

    y_pred = [int(item > threshold) for item in y_score.flatten()]
    print(np.array(y_pred).shape)#no.of test drugs cross microbes
    #print(y_pred)
    perf = evaluate_performance_two(Y_test, y_pred, y_score)
    print(perf)
    perf_list.append(perf)
print("###################################")
perf_result = get_max_avg_results(perf_list)

save_path = os.path.join("results_" + str(model_name))
version = '0506'
with open("results/" + version + '/' + save_path + "_" + version + ".json", "w") as f:
    json.dump(perf_result, f)


{'ppv': 0.0, 'accuracy': 0.9411839162904518, 'precision': 0.12358803986710963, 'recall': 0.7126436781609196, 'aupr': 0.09123865874964422, 'AUCPR': 0.41969807073503906, 'auc': 0.8281861735514495, 'f1-score': 0.21064552661381652}